In [5]:
#input
import pandas as pd
import math

In [13]:
data = {
    "x1": [0,0,0,1,0,1,0],
    "x2": [0,1,0,0,1,1,1],
    "x3": [1,0,1,0,1,0,0],
    "x4": [0,0,1,1,0,0,1],
    "y": [0,0,1,1,0,0,0]
}
data_frame = pd.DataFrame(data)
data_frame

,x1,x2,x3,x4,y
0,0,0,1,0,0
1,0,1,0,0,0
2,0,0,1,1,1
3,1,0,0,1,1
4,0,1,1,0,0
5,1,1,0,0,0
6,0,1,0,1,0


In [46]:
#calculate entropy
def entropy (data):

    entropy = 0
    data_p = len(data[data['y'] == 1])/len(data)
    data_n = len(data[data['y'] == 0])/len(data)

    if data_p != 0 and data_n != 0: # decide if the tree equal to zero
        entropy_xp = entropy = -data_p*math.log2(data_p) - data_n*math.log2(data_n)

    return entropy

In [8]:
def information_gain_fun(data):
    
    copy_data_frame = data
    len_col = len(copy_data_frame.columns)
    len_x = len_col-1

    output_table = pd.DataFrame(columns = ['factor','information_gain'])
    total_entropy = entropy(copy_data_frame)

    for i in range(len_x):
        varx = copy_data_frame.columns[i]

        xp = copy_data_frame[copy_data_frame[varx] == 1] #x = 1 data
        entropy_xp = entropy(xp) #x = 1 entropy
    
        xn = copy_data_frame[copy_data_frame[varx] == 0] #x = 0 data
        entropy_xn = entropy(xn) #x = 0 entropy

        length_total = len(copy_data_frame)
        length_xp = len(xp)
        length_xn = len(xn)

        expected_entropy = (length_xp/length_total)*entropy_xp + (length_xn/length_total)*entropy_xn
        information_gain = total_entropy - expected_entropy

        row = {'factor': varx,"information_gain":information_gain}
        output_table = pd.concat([output_table, pd.DataFrame([row])], ignore_index = True)

        #print(varx, information_gain) #is two informaiton gain, what can we do?
    return output_table

In [9]:
#tree split
def split (data, select_factor, num):
    num = num
    split_tree = data[data[select_factor] == num]
    #determine the function of y
    if split_tree['y'].nunique() == 1:
        end_point1 = [select_factor, num, int(split_tree['y'].unique())]
        #print(end_point1)
        return end_point1
    else:
        updata1 = split_tree.loc[:, split_tree.columns != select_factor]
        return updata1

In [11]:
copy_data_frame2 = data_frame
stop_positive = True
stop_negative = True

while (stop_positive or stop_negative):
    
    output_table = information_gain_fun(copy_data_frame2)
    select_value = output_table.sort_values(by = ['information_gain'], ascending = False).iloc[0]
    print(output_table)
    print(select_value)

    select_factor = select_value['factor']
    
    #positive split
    if stop_positive:
        update1 = split(copy_data_frame2, select_factor, 1) #positive split
        if isinstance(update1, list):
            print(update1)
            stop_positive = False # Stop the loop after printing
        else:
            copy_data_frame2 = update1
    
    #negative split
    if stop_negative:
        update0 = split(copy_data_frame2, select_factor, 0) #negative split
        if isinstance(update0, list):
            print(update0)
            stop_negative = False  # Stop the loop after printing
        else:
            copy_data_frame2 = update0

  factor  information_gain
0     x1          0.061743
1     x2          0.469565
2     x3          0.005978
3     x4          0.469565
factor                    x2
information_gain    0.469565
Name: 1, dtype: object
['x2', 1, 0]
  factor  information_gain
0     x1          0.251629
1     x3          0.251629
2     x4          0.918296
factor                    x4
information_gain    0.918296
Name: 2, dtype: object
['x4', 0, 0]


In [29]:
entropy(data_frame)
#calculate information gain

total_length = len(data_frame)

p_positve = len(data_frame[data_frame['y'] == 1])/len(data_frame['y'])
p_negative = len(data_frame[data_frame['y'] == 0])/len(data_frame['y'])
#print(p_positve, p_negative)
entropy = -p_positve*math.log2(p_positve) - p_negative*math.log2(p_negative)
#print(entropy)

# for x1
data_pos = data_frame[data_frame['x2'] == 1]
data_posp = len(data_pos[data_pos['y'] == 1])/len(data_pos)
data_posn = len(data_pos[data_pos['y'] == 0])/len(data_pos)

if data_posp != 0 and data_posn != 0: # decide if the tree equal to zero
    entropy_xp = -data_posp*math.log2(data_posp) - data_posn*math.log2(data_posn)
else:
    entropy_xp = 0 

data_neg = data_frame[data_frame['x2'] == 0]
data_negp = len(data_neg[data_neg['y'] == 1])/len(data_neg)
data_negn = len(data_neg[data_neg['y'] == 0])/len(data_neg)

if data_negp != 0 and data_posn != 0: # decide if the tree equal to zero
    entropy_xn = -data_negp*math.log2(data_negp) - data_negn*math.log2(data_negn)
else:
    entropy_xn = 0 


In [47]:
split_tree = data_frame[data_frame['x2'] == 0]
copy_data_frame = split_tree.loc[:, split_tree.columns != 'x2']
len_col = len(copy_data_frame.columns)
len_x = len_col-1

total_entropy = entropy(copy_data_frame)
print(total_entropy)

for i in range(len_x):
    varx = copy_data_frame.columns[i]

    xp = copy_data_frame[copy_data_frame[varx] == 1] #x = 1 data
    entropy_xp = entropy(xp) #x = 1 entropy
    

    xn = copy_data_frame[copy_data_frame[varx] == 0] #x = 0 data
    entropy_xn = entropy(xn) #x = 0 entropy

    length_total = len(copy_data_frame)
    length_xp = len(xp)
    length_xn = len(xn)
    

    expected_entropy = (length_xp/length_total)*entropy_xp + (length_xn/length_total)*entropy_xn
    information_gain = total_entropy - expected_entropy
    print(length_xp/length_total, entropy_xp, length_xn/length_total, entropy_xn, information_gain)

    #row = {'factor': varx,"information_gain":information_gain}
    #output_table = pd.concat([output_table, pd.DataFrame([row])], ignore_index = True)

    #print(varx, information_gain) #is two informaiton gain, what can we do?

0.9182958340544896
0.3333333333333333 0 0.6666666666666666 1.0 0.2516291673878229
0.6666666666666666 1.0 0.3333333333333333 0 0.2516291673878229
0.6666666666666666 0 0.3333333333333333 0 0.9182958340544896


,x1,x3,x4,y
0,0,1,0,0
2,0,1,1,1
3,1,0,1,1
